In [1]:
%aiida

Loaded AiiDA DB environment.

In [9]:

from aiida_cp2k.calculations import Cp2kCalculation

from aiida.orm import StructureData
from aiida.orm import ArrayData
from aiida.engine import submit, run_get_node

from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget, MetadataWidget, StructureBrowserWidget
from aiidalab_widgets_base import ComputerDropdown

from apps.surfaces.widgets import analyze_structure
from apps.surfaces.widgets.viewer_details import ViewerDetails

import ase
import ase.io
import numpy as np
import os
import nglview
from copy import deepcopy
from pprint import pprint
from collections import defaultdict

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from math import floor, log10

from apps.scanning_probe.hrstm.hrstm_workchain import HRSTMWorkChain
from apps.scanning_probe import common


# Select structure

In [3]:
atoms = None

def on_struct_change(c):
    global atoms
    structure = struct_browser.results.value
    if structure:
        atoms = structure.get_ase()
        atoms.pbc = [1, 1, 1]
        
        viewer_widget.setup(atoms)

    
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')

viewer_widget = ViewerDetails()

display(ipw.VBox([struct_browser, viewer_widget]))

# Select computer and codes

In [4]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

computer_drop = ComputerDropdown()

def on_computer_change(c):
    global cp2k_codes, ppm_codes, hrstm_codes
    cp2k_codes = common.comp_plugin_codes(computer_drop.selected_computer.name, 'cp2k')
    ppm_codes = common.comp_plugin_codes(computer_drop.selected_computer.name, 'spm.afm')
    hrstm_codes = common.comp_plugin_codes(computer_drop.selected_computer.name, 'spm.hrstm')
    
    drop_cp2k.options = [c.label for c in cp2k_codes]
    drop_ppm.options = [(c.label, c) for c in ppm_codes if "_2pp" in c.label]
    drop_hrstm.options = [c.label for c in hrstm_codes]
    
    
computer_drop._dropdown.observe(on_computer_change)

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_ppm = ipw.Dropdown(description="PPM code")

drop_hrstm = ipw.Dropdown(description="HR-STM code")

on_computer_change(0)

elpa_check = ipw.Checkbox(
    value=True,
    description='use ELPA',
    disabled=False
)

display(computer_drop, drop_cp2k, drop_ppm, drop_hrstm, elpa_check)

ComputerDropdown(children=(HBox(children=(Dropdown(description='Select computer:', options={'localhost': <Comp…

Dropdown(description='Cp2k code', options=('cp2k',), value='cp2k')

Dropdown(description='PPM code', options=(), value=None)

Dropdown(description='HR-STM code', options=(), value=None)

Checkbox(value=True, description='use ELPA')

# Simulation Parameters

### Probe Particle Model Parameters

In [5]:
style = {'description_width': 'initial'}
layout = {'width': '60%'}
scandx_ipw = ipw.BoundedFloatText(description="Scan d\(\mathbf{x}\) (\(Ang\))",
                                  value=0.1, min=0.05, max=0.5, step=0.05,
                                  style=style, layout=layout)
# NOTE: Minimum value of 3.0 because that is the size of the tip!
scanmin_ipw = ipw.BoundedFloatText(description="Scan \(z_{min}\) (\(Ang)\)",
                                   value=4.5, min=3.0, max=10.0, step=0.1,
                                   style=style, layout=layout)
scanmax_ipw = ipw.BoundedFloatText(description="Scan \(z_{max}\) (\(Ang\))",
                                   value=7.5, min=3.0, max=10.0, step=0.1,
                                   style=style, layout=layout)
amp_ipw = ipw.FloatText(description="Amplitude (\(Ang\))",
                        value=1.4, step=0.1,
                        style=style, layout=layout)
f0cantilever_ipw = ipw.FloatText(description="Cantilever (\(f_0\)) ",
                                 value=22352.5, step=0.1,
                                 style=style, layout=layout)
tpp_resp_ipw = ipw.ToggleButtons(description="2PP RESP model",
                            options = { # ChargeCuUp, ChargeCuDown, Ccharge, Ocharge
                                'pentacene': [-0.0669933, -0.0627402, 0.212718, -0.11767],
                                'ptcda':     [     -0.05,      -0.07,     0.23,    -0.13]
                            },
                            style=style, layout=layout)
# Show
display(scandx_ipw, scanmin_ipw, scanmax_ipw,
        amp_ipw, f0cantilever_ipw, tpp_resp_ipw)

BoundedFloatText(value=0.1, description='Scan d\\(\\mathbf{x}\\) (\\(Ang\\))', layout=Layout(width='60%'), max…

BoundedFloatText(value=4.5, description='Scan \\(z_{min}\\) (\\(Ang)\\)', layout=Layout(width='60%'), max=10.0…

BoundedFloatText(value=7.5, description='Scan \\(z_{max}\\) (\\(Ang\\))', layout=Layout(width='60%'), max=10.0…

FloatText(value=1.4, description='Amplitude (\\(Ang\\))', layout=Layout(width='60%'), step=0.1, style=Descript…

FloatText(value=22352.5, description='Cantilever (\\(f_0\\)) ', layout=Layout(width='60%'), step=0.1, style=De…

ToggleButtons(description='2PP RESP model', layout=Layout(width='60%'), options={'pentacene': [-0.0669933, -0.…

### High-Resolution STM Parameters

In [6]:
style = {'description_width': 'initial'}
layout = {'width': '60%'}
layout_min = {'width': '25%'}

voltext_ipw = ipw.Label(value="Voltage Range:",
                        style=style, layout=layout_min)
volstep_ipw = ipw.BoundedFloatText(description="Step",
                                   value=0.1, min=0.01, max=0.5, step=0.01,
                                   style=style, layout=layout_min)
volmin_ipw = ipw.FloatText(description="Min",
                           value=-0.3,
                           style=style, layout=layout_min)
volmax_ipw = ipw.FloatText(description="Max",
                           value=0.3,
                           style=style, layout=layout_min)

fwhm_ipw = ipw.BoundedFloatText(description="FWHM for DOS of Sample (\(e\)V)",
                                value=0.05, min=0.01, max=0.5, step=0.01,
                                style=style, layout=layout)
workfun_ipw = ipw.BoundedFloatText(description="Workfunction of Sample (\(e\)V)",
                                  value=5.0, min=1.0, max=10.0, step=0.1,
                                  style=style, layout=layout)
wfnstep_ipw = ipw.BoundedFloatText(description="Meshwidth for Grid Orbitals d\(\mathbf{x}\) (\(Ang\))",
                                   value=0.2, min=0.05, max=1.0, step=0.05,
                                   style=style, layout=layout)
extrap_ipw = ipw.BoundedFloatText(description="Extrapolation Plane (\(Ang\))",
                                  value=4.0, min=1.0, max=10.0, step=0.1,
                                  style=style, layout=layout)
# Tip stuff
tiptype_ipw = ipw.ToggleButtons(description="Tip Type",
                                value='blunt', options=['parametrized', 'blunt'],
                                style=style, layout=layout)
rotate_ipw = ipw.Checkbox(description="Rotate Tip Coefficients",
                          value=True,
                          style=style, layout=layout)
orbstip_ipw = ipw.BoundedIntText(description="Maximal Tip Orbital",
                                 value=1, min=0, max=1, step=1,
                                 style=style, layout=layout)
fwhmtip_ipw = ipw.BoundedFloatText(description="FWHM for DOS of Tip (\(e\)V)",
                                value=0.00, min=0.00, max=1.0, step=0.01,
                                disabled=(tiptype_ipw.value=='parametrized'),
                                style=style, layout=layout)
## Parametrized tip info
stip_ipw = ipw.BoundedFloatText(description="\(s\)-Value",
                                value=0.15, min=0.0, max=1.0, step=0.01,
                                disabled=(not tiptype_ipw.value=='parametrized'),
                                style=style, layout=layout)
pytip_ipw = ipw.BoundedFloatText(description="\(p_y\)-Value",
                                value=0.5, min=0.0, max=1.0, step=0.01,
                                disabled=(not tiptype_ipw.value=='parametrized'),
                                style=style, layout=layout)
pztip_ipw = ipw.BoundedFloatText(description="\(p_z\)-Value",
                                value=0.0, min=0.0, max=1.0, step=0.01,
                                disabled=(not tiptype_ipw.value=='parametrized'),
                                style=style, layout=layout)
pxtip_ipw = ipw.BoundedFloatText(description="\(p_x\)-Value",
                                value=0.5, min=0.0, max=1.0, step=0.01,
                                disabled=(not tiptype_ipw.value=='parametrized'),
                                style=style, layout=layout)
para_list = [stip_ipw, pytip_ipw, pztip_ipw, pxtip_ipw, fwhmtip_ipw]
def para_values(value):
    if value=='parametrized':
        stip_ipw.disabled = False
        pytip_ipw.disabled = False
        pztip_ipw.disabled = False
        pxtip_ipw.disabled = False
        fwhmtip_ipw.disabled = True
        # Always using p-orbitals for parametrized tip
        orbstip_ipw.value = 1
        orbstip_ipw.disabled = True
    else:
        stip_ipw.disabled = True
        pytip_ipw.disabled = True
        pztip_ipw.disabled = True
        pxtip_ipw.disabled = True
        fwhmtip_ipw.disabled = False
        orbstip_ipw.disabled = False
para_ipw = ipw.interactive(para_values, value=tiptype_ipw)
# Show
display(ipw.HBox([voltext_ipw,volmin_ipw,volmax_ipw,volstep_ipw], style=style, layout=layout))
display(fwhm_ipw, workfun_ipw, wfnstep_ipw, extrap_ipw,
        tiptype_ipw, rotate_ipw, orbstip_ipw, fwhmtip_ipw, stip_ipw, pytip_ipw, pztip_ipw, pxtip_ipw)

BoundedFloatText(value=0.05, description='FWHM for DOS of Sample (\\(e\\)V)', layout=Layout(width='60%'), max=…

BoundedFloatText(value=5.0, description='Workfunction of Sample (\\(e\\)V)', layout=Layout(width='60%'), max=1…

BoundedFloatText(value=0.2, description='Meshwidth for Grid Orbitals d\\(\\mathbf{x}\\) (\\(Ang\\))', layout=L…

BoundedFloatText(value=4.0, description='Extrapolation Plane (\\(Ang\\))', layout=Layout(width='60%'), max=10.…

ToggleButtons(description='Tip Type', index=1, layout=Layout(width='60%'), options=('parametrized', 'blunt'), …

...

BoundedFloatText(value=0.0, description='FWHM for DOS of Tip (\\(e\\)V)', layout=Layout(width='60%'), max=1.0,…

BoundedFloatText(value=0.15, description='\\(s\\)-Value', disabled=True, layout=Layout(width='60%'), max=1.0, …

BoundedFloatText(value=0.5, description='\\(p_y\\)-Value', disabled=True, layout=Layout(width='60%'), max=1.0,…

BoundedFloatText(value=0.0, description='\\(p_z\\)-Value', disabled=True, layout=Layout(width='60%'), max=1.0,…

BoundedFloatText(value=0.5, description='\\(p_x\\)-Value', disabled=True, layout=Layout(width='60%'), max=1.0,…

In [7]:
# Taken from AFM and slightly adjusted
def create_2pp_parameterdict(ase_geom):
    cell = ase_geom.cell
    top_z = np.max(ase_geom.positions[:, 2])
    dx = scandx_ipw.value
    resp = tpp_resp_ipw.value
    paramdict = {
        'Catom':        'Ctip',
        'Oatom':        'Otip',
        'ChargeCuUp':   resp[0],
        'ChargeCuDown': resp[1],
        'Ccharge':      resp[2],
        'Ocharge':      resp[3],
        'sigma':        0.7,
        'Cklat':        0.24600212465950813,
        'Oklat':        0.15085476515590224,
        'Ckrad':        20,
        'Okrad':        20,
        'rC0':          [0.0, 0.0, 1.82806112489999961213],
        'rO0':          [0.0, 0.0, 1.14881347770000097341],
        # We have periodic boundaries
        'PBC':          'True',
        'gridA':        list(cell[0]),
        'gridB':        list(cell[1]),
        'gridC':        list(cell[2]),
        'scanMin':      [0.0, 0.0, np.round(top_z, 1)+scanmin_ipw.value],
        'scanMax':      [cell[0,0], cell[1,1], np.round(top_z, 1)+scanmax_ipw.value],
        'scanStep':     [dx, dx, dx],
        'Amplitude':    amp_ipw.value,
        'f0Cantilever': f0cantilever_ipw.value,
        'tip':          'None',
        'Omultipole':   's',
    }
    return paramdict

In [10]:
def file_exists_func(hostname, path):
    # has to be defined in jupyter notebook because of !
    file_exists = ! ssh {hostname} "if [ -f {path} ]; then echo 1 ; else echo 0 ; fi"
    if file_exists[0] == '0':
        return False
    else:
        return True

def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not computer_drop.selected_computer:
            print("Please select a computer.")
            return

        cp2k_code = cp2k_codes[drop_cp2k.index]
        ppm_code = drop_ppm.value
        hrstm_code = hrstm_codes[drop_hrstm.index]
        
        # External folders
        parent_dir = "parent_calc_folder/"
        ppm_dir = "ppm_calc_folder/"
        
        struct = struct_browser.results.value
        ase_geom = struct.get_ase()   
        cell = ArrayData()
        cell.set_array('cell', np.diag(ase_geom.cell))
        
        # PPM parameters
        ppm_params_dict = create_2pp_parameterdict(ase_geom)
        ppm_params = Dict(dict=ppm_params_dict)
        
        # PPM folder of position
        #ppmQK = ppm_dir+"Q%1.2fK%1.2f/" %(ppm_params_dict['Ocharge'], ppm_params_dict['Oklat'])
        # new convention:
        ppmQK = ppm_dir+"Qo%1.2fQc%1.2fK%1.2f/" % (ppm_params_dict['Ocharge'], ppm_params_dict['Ccharge'],
                                                   ppm_params_dict['Oklat'])
        
        # Tip type to determine PDOS and PPM position files
        if tiptype_ipw.value != "parametrized":
            pdos_list = tiptype_ipw.value
            path = os.path.dirname(hrstm_code.get_remote_exec_path())+"/hrstm_tips/"
            pdos_list = [path+"tip_coeffs.tar.gz"]
            tip_pos = [ppmQK+"PPpos", ppmQK+"PPdisp"]
        else: # Parametrized tip
            pdos_list = [str(stip_ipw.value), str(pytip_ipw.value), 
                         str(pztip_ipw.value), str(pxtip_ipw.value)]
            tip_pos = ppmQK+"PPdisp"

        # HRSTM parameters
        hrstm_params_dict = {
            '--output':          'hrstm',
            '--voltages':        [str(val) for val in np.round(np.arange(
                volmin_ipw.value,
                volmax_ipw.value+volstep_ipw.value, 
                volstep_ipw.value), 
                len(str(volstep_ipw.min).split('.')[-1])).tolist()],
            # Sample information
            '--cp2k_input_file': parent_dir+'aiida.inp',
            '--basis_set_file':  parent_dir+'BASIS_MOLOPT',
            '--xyz_file':        parent_dir+'geom.xyz',
            '--wfn_file':        parent_dir+'aiida-RESTART.wfn',
            '--hartree_file':    parent_dir+'aiida-HART-v_hartree-1_0.cube',
            '--emin':            str(volmin_ipw.value-2.0*fwhm_ipw.value),
            '--emax':            str(volmax_ipw.value+2.0*fwhm_ipw.value),
            '--fwhm_sam':        str(fwhm_ipw.value),
            '--dx_wfn':          str(wfnstep_ipw.value),
            '--extrap_dist':     str(extrap_ipw.value),
            '--wn':              str(workfun_ipw.value),
            # Tip information
            '--pdos_list':       pdos_list,
            '--orbs_tip':        str(orbstip_ipw.value),
            '--tip_shift':       str(ppm_params_dict["rC0"][2]+ppm_params_dict["rO0"][2]),
            '--tip_pos_files':   tip_pos,
            '--fwhm_tip':        str(fwhmtip_ipw.value),
        }
        if rotate_ipw.value:
            hrstm_params_dict['--rotate'] = ''
        hrstm_params = Dict(dict=hrstm_params_dict)
   
        ## Try to access the restart-wfn file ##
        selected_comp = cp2k_code.computer
        try:
            wfn_file_path = common.find_struct_wf(struct, selected_comp, file_exists_func)
        # TODO this should catch a specific exception, not just any!
        except:
            wfn_file_path = ""
        if wfn_file_path == "":
            print("Didn't find any accessible .wfn file.")
        
        #node = submit(
        outputs, node = run_get_node(
            HRSTMWorkChain,
            cp2k_code=cp2k_code,
            structure=struct,
            cell=cell,
            wfn_file_path=Str(wfn_file_path),
            elpa_switch=Bool(elpa_check.value),
            ppm_code=ppm_code,
            ppm_params=ppm_params,
            hrstm_code=hrstm_code,
            hrstm_params=hrstm_params
        )
        # set calculation version; also used to determine post-processing
        node.set_extra("version", 0)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

Button(description='Submit', style=ButtonStyle())

Output()